# Dynamic Prompting

Dynamic prompting means constructing prompts **at runtime** by inserting variables, data, or user input — instead of hardcoding every word. This is what makes AI applications flexible and data-driven: the same code can handle thousands of different inputs.

**Why it matters:** Every real-world AI application — from chatbots to RAG systems — relies on dynamic prompts. Without them, you'd need a separate hardcoded prompt for every possible question.

**Today's journey:** Basic dynamic prompting → conversation memory → context injection patterns (which leads directly into RAG).

## Setup

!pip install openai python-dotenv -q

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found. Please create a .env file with your API key.")

client = OpenAI()

## Section 1: Static vs. Dynamic Prompts

A **static prompt** is fixed in your code. It always sends the exact same text to the model, no matter what the user wants.

In [ ]:
# Static prompt — hardcoded, inflexible
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What is photosynthesis?"}],
    temperature=0
)

print(response.choices[0].message.content)

This works, but it will **always** ask about photosynthesis. If you want to ask about something else, you have to change the code.

A **dynamic prompt** inserts variables at runtime, so the same code handles any question:

In [ ]:
# Dynamic prompt — adapts to any input
user_question = "What is machine learning?"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"Answer this question concisely: {user_question}"}],
    temperature=0
)

print(response.choices[0].message.content)

In [ ]:
# Same code, different question — no code change needed
user_question = "What is the capital of France?"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"Answer this question concisely: {user_question}"}],
    temperature=0
)

print(response.choices[0].message.content)

The API doesn't know or care whether the prompt was hardcoded or built dynamically — it just receives a string. That's the key insight: **prompts are just strings**, and we can construct them however we want.

## Section 2: Simple Dynamic Prompts

The core idea: wrap your prompt logic in a **function** that takes parameters and returns a response. This separates the data layer (variables) from the prompt layer (template).

In [ ]:
def ask(question):
    """Send a dynamic question to the model."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"Answer this question concisely: {question}"}],
        temperature=0
    )
    return response.choices[0].message.content

# Now any question works with the same function
print(ask("What is an API?"))
print("---")
print(ask("Explain recursion in one sentence."))

### Adding More Parameters

Dynamic prompts can inject more than just the question — you can control tone, format, length, or any other behavior at runtime.

In [ ]:
def ask_with_style(question, tone="professional", max_sentences=2):
    """Send a question with configurable tone and length."""
    prompt = (
        f"Answer the following question in a {tone} tone. "
        f"Keep your response to {max_sentences} sentences maximum.\n\n"
        f"Question: {question}"
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

# Same question, different styles
print("Professional:")
print(ask_with_style("What is cloud computing?", tone="professional"))
print("\nCasual:")
print(ask_with_style("What is cloud computing?", tone="casual and friendly"))

### Injecting Context

The most powerful use of dynamic prompting: injecting **external data** into the prompt so the model can answer based on information it wasn't trained on. This is the foundation of RAG.

In [ ]:
def answer_with_context(question, context):
    """Answer a question using provided context."""
    prompt = (
        f"Use the following context to answer the question. "
        f"If the context doesn't contain the answer, say so.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}"
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

# Company-specific info the model was never trained on
company_context = """
Acme Corp was founded in 2019. The CEO is Jane Smith.
Acme Corp has 150 employees and is headquartered in Berlin.
Their main product is an AI-powered inventory management system.
"""

print(answer_with_context("Who is the CEO of Acme Corp?", company_context))
print("---")
print(answer_with_context("What does Acme Corp sell?", company_context))

Notice the pattern: we build a prompt string that combines a **template** (the instructions), **context** (external data), and a **question** (user input). This is exactly what a RAG system does — except RAG retrieves the context automatically from a vector database instead of hardcoding it.

### Conversation Memory

Another form of dynamic prompting: building up the `messages` list at runtime to give the model memory of the conversation.

In [ ]:
def chat_with_memory():
    """Simple chat loop that maintains conversation history."""
    messages = [{"role": "system", "content": "You are a helpful assistant. Be concise."}]

    questions = [
        "What is Python?",
        "What are its main use cases?",  # 'its' refers to Python — the model needs memory
        "Which one is best for beginners?"
    ]

    for question in questions:
        messages.append({"role": "user", "content": question})

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )

        answer = response.choices[0].message.content
        messages.append({"role": "assistant", "content": answer})

        print(f"User: {question}")
        print(f"Assistant: {answer}\n")

chat_with_memory()

Each turn, the `messages` list grows dynamically. The model sees the full conversation history and can resolve references like "its" and "which one." This is dynamic prompting in action — the prompt is constructed at runtime from accumulated data.

## Key Takeaways

- **Prompts are just strings** — build them with f-strings, functions, or any Python logic.
- **Separate data from template** — your function takes parameters; your prompt template stays fixed.
- **Context injection** is the bridge to RAG: today we hardcode the context, but RAG retrieves it from a database.
- **Conversation memory** is another dynamic prompting pattern: the messages list grows at runtime.

**Next up:** In the RAG lesson, we'll replace the hardcoded context with automatic retrieval from a vector database — so the right information is always injected into the prompt.